# Search relevance evaluation metrics 

Consider breaking down into different types of metrics, such as precision, recall, click-through rate (CTR), and mean reciprocal rank (MRR).
Discuss the importance of each metric in measuring the relevance and effectiveness of search results.

In [1]:
"""
This code snippet will illustrate MAP metrics calculation.
We use simplified artificial data, which includes:
 1. List of documents simulating search index
 2. List of queries
 3. List of search results for each query 
"""

# 1. List of documents simulating search index. Each entry has docId, and title. We have no relevance information at this point, because index exists independsntly from queries.


documents = [
    {"doc_id": 1, "title": "Introduction to Python"},
    {"doc_id": 2, "title": "Data Structures in Java"},
    {"doc_id": 3, "title": "Web Programming with Python"},
    {"doc_id": 4, "title": "ML with TensorFlow"},
    {"doc_id": 5, "title": "Deep Learning and ML"},
    {"doc_id": 6, "title": "Natural Language Processing"},
    {"doc_id": 7, "title": "Software Programming Principles"},
    {"doc_id": 8, "title": "Database Design and SQL"},
    {"doc_id": 9, "title": "Functional Programming in Java"},
    {"doc_id": 10, "title": "Mobile App Development with Java"}
]

queries = [
    {"query_id": 1, "query": "Python programming"},
    {"query_id": 2, "query": "Understanding Java data structures"},
    {"query_id": 3, "query": "Machine Learning with TensorFlow"},
]

## Mean Average Precision (MAP)

The mean average precision (MAP) is a measure of the average relevance of search results. It is calculated by taking the average of the precision values for each query. 

Let's first define precision. Precision is a measure of the accuracy of the retrieved documents, and it is calculated as the ratio of the number of relevant documents retrieved to the total number of documents retrieved.
$$P = \frac{TP}{TP + FP}$$

Where:
- $P$ is the precision
- $TP$ is the number of true positives (relevant documents retrieved)
- $FP$ is the number of false positives (irrelevant documents retrieved)

 Let's have an example which we could compute by hand.
 For `query_id=1`, the following document are relevant: `doc_id in [1, 3, 7, 9]` because they contain at least one of query keywords: `Python`, or `Programming`.  For the search result containing `doc_id in [1, 2, 3, 4, 5, 6, 7, 8, 9]` precision is: $P=4/9=0.44$.

Now, lets define average precision $AP$). $AP$ is the average of the precision values at each rank, and it is calculated as the sum of the precision values at each relevant rank divided by the total number of relevant documents.

$$AP = \frac{1}{m} \sum_{j=1}^{m} P(j) \times rel(j)$$

Where:
- $AP$ is the average precision
- $m$ is the number of relevant documents for a given query
- $P(j)$ is the precision at position $j$
- $rel(j)$ is an indicator function that is 1 if the document at position $j$ is relevant, and 0 otherwise

Let's have an example which we could compute by hand.
Given:
- query: `query_id=1`
- search result: `doc_id in [1, 2, 3, 4, 5, 6, 7, 8, 9]`
- relevant documents: `doc_id in [1, 3, 7, 9]`
- precision at each position (also called 'rank'): 

| Position | doc_id |  Relevant| P    | AP | 
|----------|--------|----------|------|----|
| 1        | 1      |  1       | 1    | ?  | 
| 2        | 2      |  0       | 0.5  | ?  | 
| 3        | 3      |  1       | 0.67 | ?  | 
| 4        | 4      |  0       | 0.5  | ?  | 
| 5        | 5      |  0       | 0.4  | ?  | 
| 6        | 6      |  0       | 0.33 | ?  | 
| 7        | 7      |  1       | 0.43 | ?  | 
| 8        | 8      |  0       | 0.38 | ?  | 
| 9        | 9      |  1       | 0.44 | ?  | 


Finally, the mean average precision (MAP) is calculated as the average of the average precision values across all queries.

$$MAP = \frac{1}{N} \sum_{i=1}^{N} AP(i)$$

Where:
- $N$ is the number of queries
- $m$ is the number of relevant documents for a given query
- $P(j)$ is the precision at rank $j$
- $rel(j)$ is an indicator function that is 1 if the document at rank $j$ is relevant, and 0 otherwise
- The outer sum calculates the average precision for each query, and the final sum calculates the average of these average precisions across all queries.
- The MAP value ranges from 0 to 1, with 1 indicating perfect relevance and 0 indicating no relevance.

MAP is a useful metric for evaluating the overall performance of a search engine, as it takes into account the relevance of the retrieved documents for each query and provides a single score that summarizes the quality of the search results across all queries.
It is important to note that MAP is sensitive to the order of the retrieved documents, as it considers the precision at each rank. This means that the ranking of the search results can have a significant impact on the MAP score, and improving the ranking algorithm can lead to improvements in MAP.


Below we illustrate how MAP reflects the relevance. We use the same query "Python programming" and we evaluate 3 different search results.
Search result is a list of dictionaries, each dictionary has query_id, doc_id, rank, and relevance.


In [2]:
"""
MAP metric calculation
"""


def mean_average_precision(search_results, queries):
    # Create a dictionary to store the relevant documents for each query
    relevant_docs = defaultdict(list)
    for result in search_results:
        if result["relevance"] == 1:
            relevant_docs[result["query_id"]].append(result["doc_id"])

    # Calculate the average precision for each query
    average_precision = {}
    for query in queries:
        query_id = query["query_id"]
        relevant = relevant_docs[query_id]
        precision = []
        for result in search_results:
            if result["query_id"] == query_id and result["doc_id"] in relevant:
                precision.append(1)  # Document is relevant
            else:
                precision.append(0)  # Document is not relevant



In [5]:
"""
Example 1:
We have 3 different search results for the same query. Each search result has same set of documents, and relevance accurately set to 1 or 0. However ranks in each result are different, and we will demonstrate how MAP reflects the relevance. 
"""

search_result_1 = [
    # Results are set correctly, but ranks are not
    {"query_id": 1, "doc_id": 1, "rank": 1, "relevance": 1},
    {"query_id": 1, "doc_id": 2, "rank": 3, "relevance": 0},
    {"query_id": 1, "doc_id": 3, "rank": 2, "relevance": 1},  # must be ranked as top @1
    {"query_id": 1, "doc_id": 4, "rank": 4, "relevance": 0},
    {"query_id": 1, "doc_id": 5, "rank": 5, "relevance": 0},
    {"query_id": 1, "doc_id": 6, "rank": 6, "relevance": 0},
    {"query_id": 1, "doc_id": 7, "rank": 7, "relevance": 1},
    {"query_id": 1, "doc_id": 8, "rank": 8, "relevance": 0},
    {"query_id": 1, "doc_id": 9, "rank": 9, "relevance": 1},
    {"query_id": 1, "doc_id": 10, "rank": 10, "relevance": 0}
]



In [8]:
# Calculate the average precision for each query
from collections import defaultdict

# Create a dictionary to store the relevant documents for each query
relevant_docs = defaultdict(list)
for result in search_result_1:
    if result["relevance"] == 1:
        relevant_docs[result["query_id"]].append(result["doc_id"])

# Calculate the average precision for each query
average_precision = {}
for query in queries[0:2]:
    query_id = query["query_id"]
    relevant = relevant_docs[query_id]
    precision = []
    for result in search_result_1:
        if result["query_id"] == query_id and result["doc_id"] in relevant:
            precision.append(1)  # Document is relevant
        else:
            precision.append(0)  # Document is not relevant
    average_precision[query_id] = sum(precision) / len(relevant)


ZeroDivisionError: division by zero

In [10]:
relevant_docs

defaultdict(list, {1: [1], 2: [2]})